# Laboratory 8.2 Artificial Intelligence Models - Carlos García Calvo
## Investigation of a practical application of Artificial Intelligence Yolo5v

YOLO (You Only Look Once) is one of the most popular deep learning-based object detection algorithms.
Yolov5 already allows us to create and train our own object detector. It is based on a neural network that divides the image into regions, predicting identification frames and probabilities for each region; The boxes are weighted based on the predicted probabilities. The algorithm learns generalizable representations of objects, allowing a low detection error for new inputs, different from the training data set.



This program is made in a Jupyter notebook and programmed in python
Requires installing the following libraries and programs from Anaconda:

From Anaconda Terminal:

- Step 1 - installing OpenCV
pip install opencv-python
python
import cv2

- Step 2 - Check
OpenCV Version
cv2.version
exit()

In case of failure, look at https://programarfacil.com/blog/vision-artificial/instalar-opencv-python-anaconda/

- Step 3 - Installing YOLOv5 (requires Python>=3.7.0 and PyTorch>=1.7.

pip install yolov5
cd yolov5
pip install -qr https://raw.githubusercontent.com/ultralytics/yolov5/master/requirements.txt
sudo pip install scikit-learn

Webography:

ultralytics github by yolov5:
https://docs.ultralytics.com/tutorials/pytorch-hub/
https://github.com/ultralytics/yolov5

About the openCV and Jupyter Notebooks windows problem: https://medium.com/@mrdatainsight/how-to-use-opencv-imshow-in-a-jupyter-notebook-quick-tip-ce83fa32b5ad

Self-image training on Yolov5:

https://morioh.com/p/f04ccdbc3381



### This code loads a pre-trained YOLOv5s model from PyTorch Hub as a model and passes an image to an already trained yolov5s database (it is the lightest and fastest image and data recognition model).

In [ ]:
import torch
# acquisition of the Yolo5v Model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')  # o yolov5n - yolov5x6...

### We prepare libraries for the acquisition of the images/treatment, etc.

In [ ]:
import numpy as np              # maths
import cv2
from  matplotlib import pyplot as plt
%matplotlib inline


### This code loads an image directly from the webcam and displays it in a separate window:

In [ ]:
cap = cv2.VideoCapture(0)

leido, frame = cap.read()

if leido == True:
	cv2.imwrite("imagen.png", frame)
	print("Photo taken correctly and saved locally")
else:
	print("Error accessing camera")

"""
	Finally we release or release the camera
"""
cap.release()


im = cv2.imread('imagen.png',1)
im2 = im[:,:,::-1] 
plt.imshow(im2)
plt.show()

Now we can load this image saved locally or we can also load an image from the internet

In [ ]:
# we load image from the internet

img = 'imagen.png' # or file, address url, PIL, OpenCV, numpy, list...
# imagen.png
# https://ultralytics.com/images/zidane.jpg
# https://www.monederosmart.com/wp-content/uploads/2020/10/Tjw-H0m-LsD-2B6-paraguas-2-40513352_s.jpg


### We perform model search on the already trained YOLO5 neural network

In [ ]:
# Inference
results = model(img)


# Results
results.print()  # or .show(), .save(), .crop(), .pandas(), etc.
results.pandas().xyxy[0].sort_values('xmin')  # sorted left-right

In [ ]:
# WE RECORD IMAGES AND SHOW THE RESULTS OBTAINED
results.save()


In [ ]:
for i in range(0, 3):  # Update Number Based on Images Taken
    path='runs\detect\exp'+str(i)+'\imagen.jpg'
    im = cv2.imread(path,1)
    im2 = im[:,:,::-1] 
    plt.imshow(im2)
    plt.show()



### Now we're going to use YOLOv5 in a scene in Coppelia Sim so that the robot is able to classify objects by images

In [ ]:
import sim          # library to connect with CoppeliaSim
import sympy as sp  # library for symbolic calculation
import torch
import pandas
import time

import cv2 # opencv
from matplotlib import pyplot as plt
from sympy.physics.vector import init_vprinting

import numpy as np
from sympy import *

%matplotlib inline

In [ ]:
def connect(port):
# Establishes the connection to VREP
# port must match the connection port in VREP
# returns the client number or -1 if the connection cannot be established
    sim.simxFinish(-1) # just in case, close all opened connections
    clientID=sim.simxStart('127.0.0.1',port,True,True,2000,5) # Conectarse
    if clientID == 0: print("connect to", port)
    else: print("Could not connect")
    return clientID

In [ ]:
# We require the handlers for the joints, the suction cup and the suction sensor (Allows us to know if the object is close)
clientID = connect(19999)

retCode,tip=sim.simxGetObjectHandle(clientID,'suctionPadSensor',sim.simx_opmode_blocking)
retCode,suction=sim.simxGetObjectHandle(clientID,'suctionPad',sim.simx_opmode_blocking)
retCode,camera=sim.simxGetObjectHandle(clientID,'Vision_sensor',sim.simx_opmode_blocking)
retCode,joint1=sim.simxGetObjectHandle(clientID,'Joint1',sim.simx_opmode_blocking)
retCode,joint2=sim.simxGetObjectHandle(clientID,'Joint2',sim.simx_opmode_blocking)
retCode,joint3=sim.simxGetObjectHandle(clientID,'Joint3',sim.simx_opmode_blocking)
print(tip, suction, joint1, joint2, joint3)

In [ ]:
# We obtain the image from the Camera Sensor
retCode, resolution, image = sim.simxGetVisionSensorImage(clientID,camera,0,sim.simx_opmode_oneshot_wait)
img=np.array(image,dtype=np.uint8)
img.resize([resolution[1],resolution[0],3])
img = img[::-1, ::-1, :]
plt.imshow(img)
plt.show()

In [ ]:
model = torch.hub.load('ultralytics/yolov5', 'yolov5x6')

In [ ]:
# Inference
results = model(img)

# Results
positions = results.pandas().xyxy[0]
positions

In [ ]:
# I calculate the centers of each detection
positions["xcenter"] = [(positions["xmax"][i] - positions["xmin"][i])/2 + positions["xmin"][i] for i in range(len(positions))]
positions["ycenter"] = [(positions["ymax"][i] - positions["ymin"][i])/2 + positions["ymin"][i] for i in range(len(positions))]
positions

In [ ]:
# Change the centers from 0-1000 to 0-0.5 to use the map coordinates

coords = np.array([[positions["xcenter"][i]*0.5/1000, positions["ycenter"][i]*0.5/1000] for i in range(len(positions))])
positions["x"] = coords[:, 0]
positions["y"] = coords[:, 1]
positions

In [ ]:
# Delete all necessary data

detections = positions.drop(columns=["xmin", "ymin", "xmax", "ymax", "confidence", "class", "xcenter", "ycenter"])
detections

In [ ]:
# Calculate the angels of the joints with inverse kinematics
init_vprinting(use_latex='mathjax', pretty_print=False)

from sympy.physics.mechanics import dynamicsymbols
theta1, theta2, la, lb, theta, alpha, a, d = dynamicsymbols('theta1 theta2 la lb theta alpha a d')
theta1, theta2, la, lb, theta, alpha, a, d

In [ ]:
# I calculate the matrix homogenies

rot = sp.Matrix([[sp.cos(theta), -sp.sin(theta)*sp.cos(alpha), sp.sin(theta)*sp.sin(alpha)],
                 [sp.sin(theta), sp.cos(theta)*sp.cos(alpha), -sp.cos(theta)*sp.sin(alpha)],
                 [0, sp.sin(alpha), sp.cos(alpha)]])

trans = sp.Matrix([a*sp.cos(theta),a*sp.sin(theta),d])

last_row = sp.Matrix([[0, 0, 0, 1]])

m = sp.Matrix.vstack(sp.Matrix.hstack(rot, trans), last_row)

m01 = m.subs({ theta:theta1, d:0, a:la , alpha:0})
m12 = m.subs({ theta:theta2, d:0, a:lb , alpha:0})

m02 = (m01*m12)
mbee= sp.Matrix([[sp.trigsimp(m02[0,0].simplify()), sp.trigsimp(m02[0,1].simplify()), sp.trigsimp(m02[0,2].simplify()),sp.trigsimp(m02[0,3].simplify())],
                 [sp.trigsimp(m02[1,0].simplify()), sp.trigsimp(m02[1,1].simplify()), sp.trigsimp(m02[1,2].simplify()),sp.trigsimp(m02[1,3].simplify())],
                 [m02[2,0].simplify(), m02[2,1].simplify(), m02[2,2].simplify(),m02[2,3].simplify()],
                 [m02[3,0].simplify(), m02[3,1].simplify(), m02[3,2].simplify(),m02[3,3].simplify()]])

mbee

In [ ]:
def calcula(p): # Calculate IK function
    eq1 = 0.3 * cos(theta1) + 0.3 * cos(theta1 + theta2) - p[0]
    eq2 = 0.3 * sin(theta1) + 0.3 * sin(theta1 + theta2) - p[1]
    q = nsolve((eq1,eq2),(theta1,theta2),(1,1))
    return q

def mou(q): # Move joint0 & joint1  function
    retCode = sim.simxSetJointTargetPosition(clientID, joint1, q[0], sim.simx_opmode_oneshot)
    retCode = sim.simxSetJointTargetPosition(clientID, joint2, q[1], sim.simx_opmode_oneshot)
    
def setEffector(val):
# function that triggers the end effector remotely
# val is Int with value 0 or 1 to disable or activate the final actuator.
    res,retInts,retFloats,retStrings,retBuffer=sim.simxCallScriptFunction(clientID,
        "suctionPad", sim.sim_scripttype_childscript,"setEffector",[val],[],[],"", sim.simx_opmode_blocking)
    return res

def baixa(): # Down Manipulator function
    retCode = sim.simxSetJointTargetPosition(clientID, joint3, -0.095, sim.simx_opmode_oneshot)
     
def puja(): # Up Manipulator function
    retCode = sim.simxSetJointTargetPosition(clientID, joint3, 0, sim.simx_opmode_oneshot)

### In this part you have to be able to stack the cats and dogs in different places (you can use the previous functions):